In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [6]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [7]:
input_size = X_train.shape[1]
hidden_size = 16
output_size = len(set(y_train))

In [8]:
model = SimpleClassifier(input_size, hidden_size, output_size)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
batch_size = 16
epochs = 4000

for epoch in range(epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        optimizer.zero_grad()
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [100/4000], Loss: 0.1371
Epoch [200/4000], Loss: 0.0477
Epoch [300/4000], Loss: 0.0316
Epoch [400/4000], Loss: 0.0246
Epoch [500/4000], Loss: 0.0206
Epoch [600/4000], Loss: 0.0179
Epoch [700/4000], Loss: 0.0160
Epoch [800/4000], Loss: 0.0146
Epoch [900/4000], Loss: 0.0127
Epoch [1000/4000], Loss: 0.0104
Epoch [1100/4000], Loss: 0.0090
Epoch [1200/4000], Loss: 0.0079
Epoch [1300/4000], Loss: 0.0074
Epoch [1400/4000], Loss: 0.0070
Epoch [1500/4000], Loss: 0.0066
Epoch [1600/4000], Loss: 0.0065
Epoch [1700/4000], Loss: 0.0065
Epoch [1800/4000], Loss: 0.0064
Epoch [1900/4000], Loss: 0.0063
Epoch [2000/4000], Loss: 0.0060
Epoch [2100/4000], Loss: 0.0058
Epoch [2200/4000], Loss: 0.0055
Epoch [2300/4000], Loss: 0.0051
Epoch [2400/4000], Loss: 0.0048
Epoch [2500/4000], Loss: 0.0042
Epoch [2600/4000], Loss: 0.0037
Epoch [2700/4000], Loss: 0.0035
Epoch [2800/4000], Loss: 0.0032
Epoch [2900/4000], Loss: 0.0028
Epoch [3000/4000], Loss: 0.0026
Epoch [3100/4000], Loss: 0.0023
Epoch [3200/4000]

In [24]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predicted_labels = torch.argmax(test_outputs, dim=1).numpy()
    test_accuracy = accuracy_score(y_test, predicted_labels)

print(f'Test Accuracy: {test_accuracy:.4f}')

Test Accuracy: 0.9667
